In [2]:
import pandas as pd
import collections
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.sentiment import SentimentIntensityAnalyzer
import torch
from transformers import pipeline

## Dataset

In [3]:
data_df = pd.read_csv('ds_master/reddit_climatechange_dataset.csv', low_memory=False, lineterminator='\n')

In [4]:
print(len(data_df))

1954979


### VADER Sentiment analysis

In [3]:
sentiment_df = data_df

#### few preprocessing steps

In [4]:
# remove all /n characters
sentiment_df['body'] = sentiment_df['body'].str.replace('\n', '')

In [5]:
print(len(sentiment_df))

1954979


In [6]:
# remove empty values
mask = sentiment_df['body'].notnull()
sentiment_df = sentiment_df[mask]
print(len(sentiment_df))

1950768


#### VADER model

In [7]:
# Create an instance of the VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

# Define a function to compute the sentiment scores for a given text
def get_sentiment_scores(text):
    scores = analyzer.polarity_scores(text)
    return scores

# Apply the sentiment analysis function to the 'body' column of your DataFrame
sentiment_df['vader_sentiment_scores'] = sentiment_df['body'].apply(get_sentiment_scores)

# Extract the compound score from the sentiment scores and store it in a new column
sentiment_df['vader_sentiment_compound'] = sentiment_df['vader_sentiment_scores'].apply(lambda x: x['compound'])
# sentiment_df['sentiment_positive'] = sentiment_df['sentiment_scores'].apply(lambda x: x['pos'])
# sentiment_df['sentiment_negative'] = sentiment_df['sentiment_scores'].apply(lambda x: x['neg'])
# sentiment_df['sentiment_neutral'] = sentiment_df['sentiment_scores'].apply(lambda x: x['neu'])

sentiment_df['sentiment_vader'] = 'neutral'
sentiment_df.loc[sentiment_df['vader_sentiment_compound'] >= 0.05, 'sentiment_vader'] = 'positive'
sentiment_df.loc[sentiment_df['vader_sentiment_compound'] <= -0.05, 'sentiment_vader'] = 'negative'


/var/folders/6c/yrt1h_5d5974tvcf43r84dvr0000gn/T/ipykernel_59891/1570117715.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sentiment_df['vader_sentiment_scores'] = sentiment_df['body'].apply(get_sentiment_scores)
/var/folders/6c/yrt1h_5d5974tvcf43r84dvr0000gn/T/ipykernel_59891/1570117715.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sentiment_df['vader_sentiment_compound'] = sentiment_df['vader_sentiment_scores'].apply(lambda x: x['compound'])
/var/folders/6c/yrt1h_5d5974tvcf43r84dvr0000gn/T/ip

In [8]:
print(sentiment_df.head(10))

  created_utc     utc_datetime_str  subreddit       id   
0  1359582044  2013-01-30 21:40:44  AskReddit  c86hmqz  \
1  1359576987  2013-01-30 20:16:27  AskReddit  c86fpmd   
2  1359574847  2013-01-30 19:40:47  AskReddit  c86ewz9   
3  1359566007  2013-01-30 17:13:27  AskReddit  c86bu1y   
4  1359565884  2013-01-30 17:11:24  AskReddit  c86bsnj   
5  1359559914  2013-01-30 15:31:54  AskReddit  c869yg4   
6  1359532958  2013-01-30 08:02:38  AskReddit  c865vee   
7  1359526259  2013-01-30 06:10:59  AskReddit  c864ioc   
8  1359525357  2013-01-30 05:55:57  AskReddit  c864abh   
9  1359524230  2013-01-30 05:37:10  AskReddit  c863yy8   

                                                body   
0  The climate change denier analogy that I made ...  \
1  It seems to be getting more attention than tho...   
2  Build a new one in Antarctica. Underground, or...   
3  Fucking climate change. It disgusts me that so...   
4  I stress about climate change, and worry about...   
5  They can efficiently t

In [10]:
sentiment_df = sentiment_df.drop('vader_sentiment_scores', axis=1)

In [11]:
print(sentiment_df.head(10))

  created_utc     utc_datetime_str  subreddit       id   
0  1359582044  2013-01-30 21:40:44  AskReddit  c86hmqz  \
1  1359576987  2013-01-30 20:16:27  AskReddit  c86fpmd   
2  1359574847  2013-01-30 19:40:47  AskReddit  c86ewz9   
3  1359566007  2013-01-30 17:13:27  AskReddit  c86bu1y   
4  1359565884  2013-01-30 17:11:24  AskReddit  c86bsnj   
5  1359559914  2013-01-30 15:31:54  AskReddit  c869yg4   
6  1359532958  2013-01-30 08:02:38  AskReddit  c865vee   
7  1359526259  2013-01-30 06:10:59  AskReddit  c864ioc   
8  1359525357  2013-01-30 05:55:57  AskReddit  c864abh   
9  1359524230  2013-01-30 05:37:10  AskReddit  c863yy8   

                                                body   
0  The climate change denier analogy that I made ...  \
1  It seems to be getting more attention than tho...   
2  Build a new one in Antarctica. Underground, or...   
3  Fucking climate change. It disgusts me that so...   
4  I stress about climate change, and worry about...   
5  They can efficiently t

In [12]:
sentiment_df.to_csv('ds_sentiment/reddit_climatechange_dataset_sentiment.csv', index=False)

## further analysis of sentiment vader